# Importation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import healpy as hp
import yaml

import qubic
from qubic.lib.MapMaking.Qatmosphere_2d import AtmosphereMaps
from qubic.lib.Instrument.Qacquisition import QubicAcquisition
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Qscene import QubicScene
from qubic.lib.Qsamplings import get_pointing, equ2gal, QubicSampling

from qubic.lib.MapMaking.Qcg import PCGAlgorithm
from pyoperators.iterative.core import AbnormalStopIteration

from pyoperators import MPI, BlockDiagonalOperator, DiagonalOperator, ReshapeOperator

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

%matplotlib inline

# Atm class

In [ ]:
# Import simulation parameters
with open('params.yml', 'r') as file:
    params = yaml.safe_load(file) 

In [ ]:
# Call the class which build the atmosphere maps
atm = AtmosphereMaps(params)
qubic_dict = atm.qubic_dict

In [ ]:
wv_fluctuations = atm.rho_map
print(wv_fluctuations.shape[0])
plt.imshow(wv_fluctuations, cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Water vapor density fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.colorbar(label=r'$g/m^{3}$')

In [ ]:
print("Frequencies are : ", atm.frequencies, "GHz.")

In [ ]:
temp_fluctuations = np.zeros((wv_fluctuations.shape[0], wv_fluctuations.shape[1], 3))
temp_fluctuations[..., 0] = atm.get_temp_maps(wv_fluctuations)[0]
temp_fluctuations[..., 0] -= np.mean(temp_fluctuations[..., 0])

plt.imshow(temp_fluctuations[..., 0], cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Temperature fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.colorbar(label=r'$\mu K_{CMB}$')

In [ ]:
healpy_temp_fluctuations = np.zeros((hp.nside2npix(params['nside']), 3))
healpy_temp_fluctuations[..., 0] = atm.get_healpy_atm_maps_2d(atm.get_temp_maps(wv_fluctuations))[0]

index = np.where(healpy_temp_fluctuations!=0)[0]
healpy_temp_fluctuations[index, 0] -= np.mean(healpy_temp_fluctuations[index, 0])
min = np.min(healpy_temp_fluctuations[index, 0])
max = np.max(healpy_temp_fluctuations[index, 0])

hp.mollview(healpy_temp_fluctuations[..., 0], min=min, max=max, cmap='jet', title='Temperature fluctuations', unit=r'$µK_{CMB}$')
hp.gnomview(healpy_temp_fluctuations[..., 0], min=min, max=max, rot=equ2gal(0, -57), reso=15, cmap='jet', title='Temperature fluctuations', unit=r'$µK_{CMB}$')

In [ ]:
print('Angular speed', qubic_dict['angspeed'])
print('Delta azimtuh', qubic_dict['delta_az'])
print('Sweeps per elevation', qubic_dict['nsweeps_per_elevation'])
print('Angular speed psi', qubic_dict['angspeed_psi'])
print('Maximum psi', qubic_dict['maxppsi'])
print('latitude', qubic_dict['latitude'])
print('longitude', qubic_dict['longitude'])
print('Fix_azimuth', qubic_dict['fix_azimuth'])
print('period', qubic_dict['period'])
print('duration', qubic_dict['duration'])
print('Observation date', qubic_dict['date_obs'], type(qubic_dict['date_obs']))

# Scanning strategy

In [ ]:
### Random pointing
qubic_dict['random_pointing'] = False

### Sweepingpointing
qubic_dict['sweeping_pointing'] = True
qubic_dict['fix_azimuth']['apply'] = False 

qubic_dict['angspeed'] = 0.1
qubic_dict['delta_az'] = 20
qubic_dict['nsweeps_per_elevation'] = 1
qubic_dict['period'] = 10
qubic_dict['duration'] = 3
# npointings = 3600 * t_obs / period

### Repeat pointing
qubic_dict['repeat_pointing'] = False

q_sampling = get_pointing(qubic_dict)
print(q_sampling)

In [ ]:
plt.plot(q_sampling.azimuth, label="Azimuth")
plt.plot(q_sampling.elevation, label="Elevation")
plt.legend()
plt.ylabel('Angle (deg)')
plt.xlabel('Pointing')

plt.figure()
plt.plot(q_sampling.azimuth, q_sampling.elevation, 'o')
plt.xlabel('Azimuth (deg)')
plt.ylabel('Elevation (deg)')

In [ ]:
from pysimulators.interfaces.healpy import Spherical2HealpixOperator
from pysimulators import SphericalHorizontal2EquatorialOperator, SphericalEquatorial2GalacticOperator
from astropy.time import Time, TimeDelta

def _format_sphconv(a, b, date_obs=None, time=None):
    incoords = np.empty(np.broadcast(a, b).shape + (2,))
    incoords[..., 0] = a
    incoords[..., 1] = b
    if date_obs is None:
        return incoords
    time = Time(date_obs, scale='utc') + TimeDelta(time, format='sec')
    return incoords, time

incoords, time = _format_sphconv(q_sampling.azimuth, q_sampling.elevation, date_obs=qubic_dict['date_obs'], time=q_sampling.time)
h2e = SphericalHorizontal2EquatorialOperator(
        'NE', time, qubic_dict['latitude'], qubic_dict['longitude'], degrees=True)
e2g = SphericalEquatorial2GalacticOperator(degrees=True)
outcoords = e2g(h2e(incoords))

In [ ]:
test = np.zeros(hp.nside2npix(params['nside']))
azel = np.asarray([q_sampling.azimuth, q_sampling.elevation]).T
index = np.array(Spherical2HealpixOperator(params['nside'], 'azimuth, elevation')(np.radians(outcoords)), dtype='int')
test[index] = 1
print(len(index))
hp.mollview(test, title="Sweeping scanning strategy")
hp.gnomview(test, rot=equ2gal(0, -57), reso=15, title="Sweeping scanning strategy")

# Wind

In [ ]:
def random_wind(npointing, wind_mean, wind_std):
    wind_x = np.random.normal(wind_mean, wind_std, npointing)
    wind_y = np.random.normal(wind_mean, wind_std, npointing)
    
    return wind_x, wind_y

def constant_wind(npointing, wind_x, wind_y):
    ones = np.cumsum(np.ones(npointing))
    return wind_x*ones, wind_y*ones

def random_wind_corrected(npointing, wind_mean, wind_std):
    
    wind_x = np.cumsum(np.random.normal(wind_mean, wind_std, npointing))
    wind_y = np.cumsum(np.random.normal(wind_mean, wind_std, npointing))
    
    return wind_x, wind_y

In [ ]:
# wind_x, wind_y = random_wind_corrected(len(q_sampling.index), 0, 5)
wind_x, wind_y = constant_wind(len(q_sampling.index), -1,0)

plt.plot(wind_x, wind_y, '.')

In [ ]:
def azel_to_cartesian(azimuth, elevation, altitude):
    x = altitude / np.sin(elevation) * np.cos(elevation) * np.cos(azimuth)
    y = altitude / np.sin(elevation) * np.cos(elevation) * np.sin(azimuth)
    return x, y

def cartesian_to_azel(x, y ,altitude):
    r = np.sqrt(x**2 + y**2 + altitude**2)
    el = np.pi/2 - np.arccos(altitude/r)
    az = np.arctan2(y, x)
    return az, el

In [ ]:
x, y = azel_to_cartesian(np.radians(q_sampling.azimuth), np.radians(q_sampling.elevation), params['altitude_atm_2d'])
plt.plot(x, y, 'k', label='Scanning Strategy', markersize=2)
plt.imshow(temp_fluctuations[..., 0], cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Water vapor density fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.legend()
# plt.xlim(-1100, -600)
# plt.ylim(-800, -200)
plt.colorbar(label=r'$g/m^{3}$')

In [ ]:
# Attention, il faut que j'ajoute le fait que le vent est en m/s, et qu'il y a du temps entre chaque point,
# pour pouvoir calculer correctement le décalage dû au vent

def get_deviation_index(position_x, position_y, wind_x, wind_y, delta_time):
    deviated_index_x = (np.round(wind_x) + np.round(position_x)).astype(int)
    deviated_index_y = (np.round(wind_y) + np.round(position_y)).astype(int)
    return deviated_index_x, deviated_index_y
delta_time = qubic_dict['duration']*3600/qubic_dict['period']
deviated_index_x, deviated_index_y = get_deviation_index(x, y, wind_x, wind_y, delta_time)

In [ ]:
plt.plot(x, y, label='cartesian position')
plt.plot(deviated_index_x, deviated_index_y, label='deviated cartesian position')
plt.legend()

In [ ]:
plt.figure()
plt.plot(x, y, 'k', label='Scanning Strategy', markersize=2)
plt.plot(deviated_index_x, deviated_index_y, 'or', label='Deviated Scanning Strategy', markersize=2)
plt.imshow(temp_fluctuations[..., 0], cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Water vapor density fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.legend()
plt.colorbar(label=r'$g/m^{3}$')

plt.figure()
plt.plot(x, y, 'k', label='Scanning Strategy', markersize=2)
plt.plot(deviated_index_x, deviated_index_y, 'or', label='Deviated Scanning Strategy', markersize=2)
plt.imshow(temp_fluctuations[..., 0], cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Water vapor density fluctuations - Zoom')
plt.xlabel('m')
plt.ylabel('m')
plt.xlim(-3000, -600)
plt.ylim(-1800, -200)
plt.legend() 
plt.colorbar(label=r'$g/m^{3}$')

In [ ]:
deviated_az, deviated_el = cartesian_to_azel(deviated_index_x, deviated_index_y, params['altitude_atm_2d'])
deviated_az, deviated_el = np.degrees(deviated_az%(2*np.pi)), np.degrees(deviated_el)

plt.plot(q_sampling.azimuth, label='original')
plt.plot(deviated_az, label='deviated')
plt.legend()
plt.title("Azimuth")
plt.xlabel("Sample")
plt.ylabel("Azimuth (Degrees)")

plt.figure()
plt.plot(q_sampling.elevation, label='original')
plt.plot(deviated_el, label='deviated')
plt.legend()
plt.title("Elevation")
plt.xlabel("Sample")
plt.ylabel("Elevation (Degrees)")

plt.figure()
plt.plot(q_sampling.azimuth, q_sampling.elevation, 'k', label='original')
plt.plot(deviated_az, deviated_el, 'r', label='deviated')
plt.xlabel('Azimuth (Degrees)')
plt.ylabel('Elevation (Degrees)')
plt.legend()
plt.title('Scanning Strategy')

# Create deviated Scanning Strategy 

IMPORTANT: To perform MM on CMB + atm, we will have to take care to avoid to apply the deviated scanning strategy on the CMB. One idea can be to create two acquision operators: one for the CMB (and other astrophysical sources) with the standard scanning strategy and one for the atmosphere with the deviated scanning strategy.

In [ ]:
q_sampling

In [ ]:
q_sampling_deviated = QubicSampling(azimuth=deviated_az, elevation=deviated_el, pitch=q_sampling.pitch, 
                                    angle_hwp=q_sampling.angle_hwp, time=q_sampling.time, 
                                    period=q_sampling.period, latitude=q_sampling.latitude, longitude=q_sampling.longitude)

q_sampling_deviated.fix_az = q_sampling.fix_az

In [ ]:
q_sampling_deviated

In [ ]:
plt.plot(q_sampling.azimuth, q_sampling.elevation, 'k', label='original')
plt.plot(q_sampling_deviated.azimuth, q_sampling_deviated.elevation, 'r', label='deviated')
plt.xlabel('Azimuth (Degrees)')
plt.ylabel('Elevation (Degrees)')
plt.legend()
plt.title('Scanning Strategy')

# Build MM

## Build QUBIC instance

In [ ]:
q_instrument = QubicInstrument(qubic_dict)
q_scene = QubicScene(qubic_dict)

q_acquisition = QubicAcquisition(q_instrument, q_sampling, q_scene, qubic_dict)
q_acquisition_deviated = QubicAcquisition(q_instrument, q_sampling_deviated, q_scene, qubic_dict)

In [ ]:
coverage = q_acquisition.get_coverage()

covnorm = coverage / coverage.max()
seenpix = covnorm > params['coverage_cut']

center = np.array([0, -57])
qubic_patch = qubic.lib.Qsamplings.equ2gal(center[0], center[1])

coverage_deviated = q_acquisition_deviated.get_coverage()

covnorm_deviated = coverage_deviated / coverage_deviated.max()
seenpix_deviated = covnorm_deviated > params['coverage_cut']

## Compare (azimuth, elevation) coordinates

In [ ]:
# Before wind
incoords, time = _format_sphconv(q_sampling.azimuth, q_sampling.elevation, date_obs=qubic_dict['date_obs'], time=q_sampling.time)
h2e = SphericalHorizontal2EquatorialOperator(
        'NE', time, qubic_dict['latitude'], qubic_dict['longitude'], degrees=True)
e2g = SphericalEquatorial2GalacticOperator(degrees=True)
outcoords = e2g(h2e(incoords))
index = np.array(Spherical2HealpixOperator(params['nside'], 'azimuth, elevation')(np.radians(outcoords)), dtype='int')
pointing_before_wind = np.zeros(hp.nside2npix(params['nside']))
pointing_before_wind[index] = 1

# After wind
incoords_deviated, time = _format_sphconv(q_sampling_deviated.azimuth, q_sampling_deviated.elevation, date_obs=qubic_dict['date_obs'], time=q_sampling.time)
h2e = SphericalHorizontal2EquatorialOperator(
        'NE', time, qubic_dict['latitude'], qubic_dict['longitude'], degrees=True)
e2g = SphericalEquatorial2GalacticOperator(degrees=True)
outcoords_deviated = e2g(h2e(incoords_deviated))
index = np.array(Spherical2HealpixOperator(params['nside'], 'azimuth, elevation')(np.radians(outcoords_deviated)), dtype='int')
pointing_after_wind = np.zeros(hp.nside2npix(params['nside']))
pointing_after_wind[index] = 1

In [ ]:
hp.mollview(pointing_before_wind, title="Scanning Strategy")
hp.mollview(pointing_after_wind, title="Deviated Scanning Strategy")

In [ ]:
hp.gnomview(pointing_before_wind, rot=equ2gal(0, -57), reso=15, title="Scanning strategy")
hp.gnomview(pointing_after_wind, rot=equ2gal(0, -57), reso=15, title="Deviated Scanning strategy")

## Compare coverage

In [ ]:
hp.gnomview(coverage, rot=equ2gal(0, -57), reso=15, title="Coverage")
hp.gnomview(coverage_deviated, rot=equ2gal(0, -57), reso=15, title="Deviated Coverage")

## Build QUBIC operators

In [ ]:
H_qubic_deviated = q_acquisition_deviated.get_operator()
H_qubic = q_acquisition.get_operator()
R_qubic = ReshapeOperator(H_qubic.shapeout, H_qubic.shape[0])
invN_qubic = R_qubic(q_acquisition_deviated.get_invntt_operator(False, False)(R_qubic.T))
d_qubic = H_qubic_deviated(healpy_temp_fluctuations).ravel()

In [ ]:
H = R_qubic(H_qubic)
invN = invN_qubic
d = d_qubic

## Map-Making

In [ ]:
A = H.T * invN * H
b = H.T * invN * d.flatten()
x0 = healpy_temp_fluctuations*0
true_maps = healpy_temp_fluctuations

In [ ]:
stacked_dptdp_inv = np.zeros((1,12*params['nside']**2))

D = H.operands[2]
P = H.operands[-1]
sh = P.matrix.data.index.shape
no_det = 992
point_per_det = int(sh[0] / no_det)
mapPtP_perdet_seq = np.zeros((no_det, 12 * params['nside']**2))
sample_ranges = [(det * point_per_det, (det + 1) * point_per_det) for det in range(no_det)]
for det, (start, end) in enumerate(sample_ranges):
    indices = P.matrix.data.index[start:end, :]  
    weights = P.matrix.data.r11[start:end, :]
    flat_indices = indices.ravel()
    flat_weights = weights.ravel()

    mapPitPi = np.zeros(12 * params['nside']**2)
    np.add.at(mapPitPi, flat_indices, flat_weights**2)

    mapPtP_perdet_seq[det, :] = mapPitPi
D_elements = D.data
D_sq = D_elements**2
mapPtP_seq_scaled = D_sq[:, np.newaxis] * mapPtP_perdet_seq 
dptdp = mapPtP_seq_scaled.sum(axis = 0)
dptdp_inv = 1 / dptdp
dptdp_inv[np.isinf(dptdp_inv)] = 0.
stacked_dptdp_inv[0] = dptdp_inv
M = BlockDiagonalOperator( \
            [DiagonalOperator(ci, broadcast='rightward') for ci in stacked_dptdp_inv],
            new_axisin=0)

# Map-Making

In [ ]:
# Run PCG
algo = PCGAlgorithm(
    A,
    b,
    comm,
    x0=x0,
    tol=1e-10,
    maxiter=200,
    disp=True,
    M=M,
    center=[0, -57],
    reso=15,
    seenpix=seenpix,
    input=true_maps,
)
try:
    output = algo.run()
    success = True
    message = 'Success'
except AbnormalStopIteration as e:
    output = algo.finalize()
    success = False
    message = str(e)

In [ ]:
plt.plot(output['convergence'])
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Convergence')

In [ ]:
plt.figure(figsize=(10, 12), dpi=200)
k=1
# true_maps[~seenpix, :] = hp.UNSEEN
# output['x'][~seenpix, :] = hp.UNSEEN

istk = 0
n_sig = 3
reso = 15

sigma = np.std(true_maps[seenpix, istk])
hp.gnomview(true_maps[:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', rot=qubic_patch, reso=reso, sub=(1, 3, k), title='Input', notext=True)
hp.gnomview(output['x'][:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', rot=qubic_patch, reso=reso, sub=(1, 3, k+1), title='Output', notext=True)
hp.gnomview(output['x'][:, istk] - true_maps[:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', rot=qubic_patch, reso=reso, sub=(1, 3, k+2), title='Residual', notext=True)
k+=3

In [ ]:
plt.figure(figsize=(12, 25))
k=1
true_maps[~seenpix, :] = hp.UNSEEN
output['x'][~seenpix, :] = hp.UNSEEN

istk = 0
n_sig = 3

sigma = np.std(true_maps[seenpix, istk])
hp.mollview(true_maps[:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k), title='Input', notext=True)
hp.mollview(output['x'][:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k+1), title='Output', notext=True)
hp.mollview(output['x'][:, istk] - true_maps[:, istk], cmap='jet', sub=(1, 3, k+2), title='Residual', notext=True)
k+=3

In [ ]:
plt.figure(figsize=(10, 12), dpi=200)
k=1
true_maps[~seenpix, :] = hp.UNSEEN
output['x'][~seenpix, :] = hp.UNSEEN

istk = 0
n_sig = 3

sigma = np.std(true_maps[seenpix, istk])
hp.gnomview(true_maps[:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', rot=qubic_patch, reso=15, sub=(1, 3, k), title='Input', notext=True)
hp.gnomview(output['x'][:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', rot=qubic_patch, reso=15, sub=(1, 3, k+1), title='Output', notext=True)
hp.gnomview(output['x'][:, istk] - true_maps[:, istk], cmap='jet', rot=qubic_patch, reso=15, sub=(1, 3, k+2), title='Residual', notext=True)
k+=3